In [1]:
import numpy as np
import matplotlib.pyplot as plt
import deepdish as dd

### load data

In [4]:
vector_field = dd.io.load('vector_fields.h5')

In [5]:
vector_field.shape

(105, 4, 840, 840)

### create abs distance matrices

In [6]:
abs_distance = np.zeros_like(vector_field)

### 4 channels meaning: (dxh, dyh, dxt, dyt)

In [7]:
abs_distance.shape

(105, 4, 840, 840)

### compute abs distance

In [22]:
def compute_abs_dist(abs_distance):
    pad_distance = np.arange(abs_distance.shape[2])
    n_repeat = abs_distance.shape[2]
    n_data = abs_distance.shape[0]
    
    for i in range(n_data):
        abs_distance[i][1] = abs_distance[i][3] = np.repeat(pad_distance.reshape(-1,1), n_repeat, axis=1)
        abs_distance[i][0] = abs_distance[i][2] = np.repeat(pad_distance.reshape(1,-1), n_repeat, axis=0)
    
    return abs_distance

In [23]:
abs_distance = compute_abs_dist(abs_distance)

### add abs distance to vector field

In [24]:
abs_vf = vector_field + abs_distance

### compare abs_vf location with labels in labeled data e.g. labeled_data[0] and abs_vf[0]

In [86]:
labeled_data = dd.io.load('clean_labeled_data.h5')

In [87]:
labeled_data.shape

(105, 840, 840)

In [229]:
# get abs_vf key point indeces
def get_indeces(matrix):
    n_channel = matrix.shape[0]
    x_indeces = []
    y_indeces = []
    
    x_indeces.append(np.concatenate((np.unique(matrix[0]), np.unique(matrix[2]))))
    y_indeces.append(np.concatenate((np.unique(matrix[1]), np.unique(matrix[3]))))
    
    return np.unique(np.array(x_indeces).reshape(1,-1)).astype(np.int64), np.unique(np.array(y_indeces).reshape(1,-1)).astype(np.int64)

### verify if abs+vector_field is equivelent to coordinates of key_points for origional dataset

In [239]:
def verify_identical(labeled_data, abs_vf):
    n_data = len(abs_vf)
    n_error = 0
    
    for i in range(n_data):
        labeled_indeces = np.where(labeled_data[i]!=0)
        x_indeces, y_indeces = get_indeces(abs_vf[i])
        if np.setdiff1d(np.unique(labeled_indeces[0]), x_indeces).size!=0 or np.setdiff1d(np.unique(labeled_indeces[1]), y_indeces).size!=0:
            print(i)
            n_error += 1
    
    if n_error == 0:
        print('all abs+vf are identical to indeces in labeled_data!')

In [240]:
verify_identical(labeled_data, abs_vf)

all abs+vf are identical to indeces in labeled_data!
